# Search Engine for Jupyter notebooks
## Creating a dictionary from the notebook files

In [165]:
# import
import json
import os
from collections import defaultdict
import numpy as np

In [4]:
folder = 'PythonDataScienceHandbook-notebooks'
files = [file for file in os.listdir(os.getcwd()+'\\'+folder) if file[-6:]=='.ipynb']
files

['00.00-Preface.ipynb',
 '01.00-IPython-Beyond-Normal-Python.ipynb',
 '01.01-Help-And-Documentation.ipynb',
 '01.02-Shell-Keyboard-Shortcuts.ipynb',
 '01.03-Magic-Commands.ipynb',
 '01.04-Input-Output-History.ipynb',
 '01.05-IPython-And-Shell-Commands.ipynb',
 '01.06-Errors-and-Debugging.ipynb',
 '01.07-Timing-and-Profiling.ipynb',
 '01.08-More-IPython-Resources.ipynb',
 '02.00-Introduction-to-NumPy.ipynb',
 '02.01-Understanding-Data-Types.ipynb',
 '02.02-The-Basics-Of-NumPy-Arrays.ipynb',
 '02.03-Computation-on-arrays-ufuncs.ipynb',
 '02.04-Computation-on-arrays-aggregates.ipynb',
 '02.05-Computation-on-arrays-broadcasting.ipynb',
 '02.06-Boolean-Arrays-and-Masks.ipynb',
 '02.07-Fancy-Indexing.ipynb',
 '02.08-Sorting.ipynb',
 '02.09-Structured-Data-NumPy.ipynb',
 '03.00-Introduction-to-Pandas.ipynb',
 '03.01-Introducing-Pandas-Objects.ipynb',
 '03.02-Data-Indexing-and-Selection.ipynb',
 '03.03-Operations-in-Pandas.ipynb',
 '03.04-Missing-Values.ipynb',
 '03.05-Hierarchical-Indexing.ip

In [86]:
# def read_ipynb(file,folder):
#     cwd = os.getcwd()
#     with open(cwd+'\\'+folder+'\\'+file,encoding="utf8") as file:
#         data = json.load(file)
#         code = []
#         markdown = []
#         for cell in data['cells']:
#             clean_cell = list(map(lambda s: s.strip(), cell['source'])) #remove the '\n' at the end of each string in the list
#             if cell['cell_type'] == 'markdown':
#                 markdown += clean_cell
#             if cell['cell_type'] == 'code':
#                 code += clean_cell
                
#     markdown_str = ' '.join(map(str, markdown))
#     code_str = ' '.join(map(str, code))
#     return markdown,code,markdown_str,code_str
        

# # read_ipynb('00.00-Preface.ipynb',folder)

In [5]:
def read_ipynb(file,folder):
    cwd = os.getcwd()
    with open(cwd+'\\'+folder+'\\'+file,encoding="utf8") as file:
        data = json.load(file)
        code = []
        markdown = []
        modules = []
        heading = []
        code_output = []
        
        output_line = []
        for cell in data['cells']:
            clean_cell = list(map(lambda s: s.strip(), cell['source'])) #remove the '\n' at the end of each string in the list         
            for line in clean_cell:
                if line[:6]=='import' or line[:4]=='from':
                    modules += [line]
                if line[:1] == '#':
                    heading += [line]
            
            if cell['cell_type'] == 'markdown':
                markdown += clean_cell
            
            if cell['cell_type'] == 'code':
                code += clean_cell
                if cell['outputs']!=[]:
                    output_type = cell['outputs'][0]['output_type']
                    if output_type == 'stream':
                        output_line = cell['outputs'][0]['text']
                    if output_type == 'execute_result':
                        output_line = cell['outputs'][0]['data']['text/plain']
                    code_output += output_line

    markdown_str = ' '.join(map(str, markdown))
    code_str = ' '.join(map(str, code))
    code_output_str = ' '.join(map(str, code_output))
    modules = list(set(modules))

    return sorted(modules),heading,markdown_str,code_str,code_output_str #markdown,code

In [9]:
%%time
read_ipynb('02.02-The-Basics-Of-NumPy-Arrays.ipynb',folder)

Wall time: 2 ms


(['import numpy as np'],
 ['# The Basics of NumPy Arrays',
  '## NumPy Array Attributes',
  '## Array Indexing: Accessing Single Elements',
  '## Array Slicing: Accessing Subarrays',
  '### One-dimensional subarrays',
  '### Multi-dimensional subarrays',
  '#### Accessing array rows and columns',
  '### Subarrays as no-copy views',
  '### Creating copies of arrays',
  '## Reshaping of Arrays',
  '# row vector via reshape',
  '# row vector via newaxis',
  '# column vector via reshape',
  '# column vector via newaxis',
  '## Array Concatenation and Splitting',
  '### Concatenation of arrays',
  '# concatenate along the first axis',
  '# concatenate along the second axis (zero-indexed)',
  '# vertically stack the arrays',
  '# horizontally stack the arrays',
  '### Splitting of arrays'],
 '<!--BOOK_INFORMATION--> <img align="left" style="padding-right:10px;" src="figures/PDSH-cover-small.png">  *This notebook contains an excerpt from the [Python Data Science Handbook](http://shop.oreilly.

In [42]:
%%time

ipynb_dict = defaultdict()
for file in files:
    temp_dict = {}
    values = read_ipynb(file,folder)
    temp_dict['file_name'] = file
    temp_dict['modules'] = values[0]
    temp_dict['heading'] = values[1]
    temp_dict['markdown_str'] = values[2]
    temp_dict['code_str'] = values[3]
    temp_dict['code_output_str'] = values[4]
    ipynb_dict[file] = temp_dict

Wall time: 805 ms


In [43]:
ipynb_dict['02.02-The-Basics-Of-NumPy-Arrays.ipynb']['modules']

['import numpy as np']

## Analysing the dictionary by creating a index

### Create a dataframe

In [47]:
notebooks_df = pd.DataFrame.from_dict(ipynb_dict,orient='index').reset_index(drop=True)
notebooks_df

,file_name,modules,heading,markdown_str,code_str,code_output_str
0,00.00-Preface.ipynb,[],"[# Preface, ## What Is Data Science?, ## Who I...","<!--BOOK_INFORMATION--> <img align=""left"" styl...",,
1,01.00-IPython-Beyond-Normal-Python.ipynb,[],"[# IPython: Beyond Normal Python, ## Shell or ...","<!--BOOK_INFORMATION--> <img align=""left"" styl...",,
2,01.01-Help-And-Documentation.ipynb,[],"[# Help and Documentation in IPython, ## Acces...","<!--BOOK_INFORMATION--> <img align=""left"" styl...",,
3,01.02-Shell-Keyboard-Shortcuts.ipynb,[],"[# Keyboard Shortcuts in the IPython Shell, ##...","<!--BOOK_INFORMATION--> <img align=""left"" styl...",,
4,01.03-Magic-Commands.ipynb,[],"[# IPython Magic Commands, ## Pasting Code Blo...","<!--BOOK_INFORMATION--> <img align=""left"" styl...",,
...,...,...,...,...,...,...
63,05.13-Kernel-Density-Estimation.ipynb,"[from mpl_toolkits.basemap import Basemap, fro...","[# In-Depth: Kernel Density Estimation, ## Mot...","<!--BOOK_INFORMATION--> <img align=""left"" styl...",%matplotlib inline import matplotlib.pyplot as...,"1.0 1.0 (-0.2, 8) (-0.2, 8) (-0.2, 8) (-0.02, ..."
64,05.14-Image-Features.ipynb,"[from itertools import chain, from skimage imp...","[# Application: A Face Detection Pipeline, ## ...","<!--BOOK_INFORMATION--> <img align=""left"" styl...",%matplotlib inline import matplotlib.pyplot as...,"(13233, 62, 47) (30000, 62, 47) (30000, 62, 47..."
65,05.15-Learning-More.ipynb,[],"[# Further Machine Learning Resources, ## Mach...","<!--BOOK_INFORMATION--> <img align=""left"" styl...",,
66,06.00-Figure-Code.ipynb,"[from helpers_05_08 import visualize_tree, fro...","[# Appendix: Figure Code, ## Broadcasting, # A...","<!--BOOK_INFORMATION--> <img align=""left"" styl...",%matplotlib inline import matplotlib.pyplot as...,Overwriting helpers_05_08.py\n Overwriting hel...


## Elastic Search Setup Local

In [14]:
# import
from elasticsearch import Elasticsearch # <== May need to pip install this
import pandas as pd
from tqdm import tqdm_notebook

In [50]:
HOST = 'http://localhost:9200/'
es = Elasticsearch(hosts=[HOST])

INDEX="handboek"
TYPE= "record"

def rec_to_actions(df):
    for record in df.to_dict(orient="records"):
        yield ('{ "index" : { "_index" : "%s", "_type" : "%s" }}'% (INDEX, TYPE))
        yield (json.dumps(record, default=int))

### Put the dataframe ..

In [51]:
# to dataframe
# df = pd.read_dict(wiki_dict,orient='index')


def index_marks(nrows, chunk_size):
    return range(1 * chunk_size, (nrows // chunk_size + 1) * chunk_size, chunk_size)


def split(dfm, chunk_size):
    indices = index_marks(dfm.shape[0], chunk_size)
    return np.split(dfm, indices) 

# split up the dataframe
chunks = split(notebooks_df, 9000)

# Now bulk index all the chunks
c = len(chunks)
for c in tqdm_notebook(chunks):
    if c.shape[0]>0:
        r = es.bulk(rec_to_actions(c)) # return a dict
print('Done')


Done


In [52]:
# !curl "http://localhost:9200/handboek"
# !curl -XDELETE "localhost:9200/handboek"

!curl "http://localhost:9200/_cat/indices?v"


# !curl -XPOST "http://localhost:9200/_shutdown"

health status index    uuid                   pri rep docs.count docs.deleted store.size pri.store.size
yellow open   handboek 8SoUSayXT_OGv2TXxjdXhQ   1   1         68            0     16.2kb         16.2kb


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   208  100   208    0     0  10947      0 --:--:-- --:--:-- --:--:-- 10947


In [350]:
def multi_match_query(code,markdown,modules,highlight):
    query_str = ''
    fields = []
    
    if code != None:
        fields += ['code_str']
        query_str += ' '+code
    if markdown != None:
        fields += ['markdown_str']
        query_str += ' '+markdown
    if modules != None:
        fields += ['modules']
        query_str += ' '+modules   
    
        
    q = {}
    query_dict = {}
    multi_match = {}
    
    multi_match['fields'] = fields
    multi_match['query'] = query_str
    query_dict['multi_match'] = multi_match
    q['query'] = query_dict
    
    if highlight != None:
        highlight = {"pre_tags":["<b>"],
            "post_tags":["</b>"],
            "fields":{'markdown_str':{}}}
        q['highlight'] = highlight
    return q

def query_string_query(code,markdown,modules,highlight):
    query_str = ''
    fields = []
    
    if code != None:
        fields += ['code_str']
        query_str += '(code_str:'+code+')'
    if markdown != None:
        fields += ['markdown_str']
        if query_str != '':
            query_str += ' AND '
        query_str += '(markdown_str:'+markdown+')'
    if modules != None:
        fields += ['modules']
        if query_str != '':
            query_str += ' AND '
        query_str += '(modules:'+modules+')'  
    
        
    q = {}
    query_dict = {}
    query_string = {}
    
    query_string['query'] = query_str
    query_dict['query_string'] = query_string
    q['query'] = query_dict
    
    if highlight == True:
        highlight = {"pre_tags":["<b>"],
                     "post_tags":["</b>"],
                     "order":"score",
                     "fields":{'markdown_str':{},'code_str':{}}}
        q['highlight'] = highlight
    return q

# "order":"score",
# "fields":{'_all':{}}} WERKT NIET....


# https://www.elastic.co/guide/en/elasticsearch/reference/6.8/search-request-highlighting.html

In [ ]:
# https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl-multi-match-query.html

# {
#   "query": {
#     "multi_match" : {
#       "query":    "this is a test", 
#       "fields": [ "subject", "message" ] 
#     }
#   }
# }

In [ ]:
# https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl-query-string-query.html

# {
#     "query": {
#         "query_string": {
#             "query": "(content:this OR name:this) AND (content:that OR name:that)"
#         }
#     }
# }

In [276]:
q_test_multimatch = multi_match_query('linear',None,'numpy',True)
q_test_multimatch

{'query': {'multi_match': {'fields': ['code_str', 'modules'],
   'query': ' linear numpy'}},
 'highlight': {'pre_tags': ['<b>'],
  'post_tags': ['</b>'],
  'fields': {'markdown_str': {}}}}

In [343]:
q_test_querystring = query_string_query(None,'drop','pandas',True)
q_test_querystring

{'query': {'query_string': {'query': '(markdown_str:drop) AND (modules:pandas)'}},
 'highlight': {'pre_tags': ['<b>'],
  'post_tags': ['</b>'],
  'order': 'score',
  'fields': {'_all': {}}}}

In [181]:
# def search_results(query,max_res):
#     result = es.search(body=query, size=max_res)
#     transcripts = ""
#     for i in range(len(result['hits']['hits'])): # for all hits
#         res = result['hits']['hits'][i]
#         modules = res['_source']['modules']
#         #len(res['_source']['modules'])
#         print(res['_score'],'https://github.com/jakevdp/PythonDataScienceHandbook/blob/master/notebooks/'+res['_source']['file_name'])
#         print(res['highlight']['markdown_str'])

In [285]:
# q_test = multi_match_query(None,'linear regression',None,False)
# print(q_test)
# # print()

# search_results(q_test,20)

In [109]:
result['hits']['hits'][0]

{'_index': 'handboek',
 '_type': 'record',
 '_id': 'LTR2YnABgBRbd-7Oab31',
 '_score': 4.1406717,
 '_source': {'file_name': '05.07-Support-Vector-Machines.ipynb',
  'modules': ['from ipywidgets import interact, fixed',
   'from mpl_toolkits import mplot3d',
   'from scipy import stats',
   'from sklearn.cross_validation import train_test_split',
   'from sklearn.datasets import fetch_lfw_people',
   'from sklearn.datasets.samples_generator import make_blobs',
   'from sklearn.datasets.samples_generator import make_circles',
   'from sklearn.decomposition import RandomizedPCA',
   'from sklearn.grid_search import GridSearchCV',
   'from sklearn.metrics import classification_report',
   'from sklearn.metrics import confusion_matrix',
   'from sklearn.pipeline import make_pipeline',
   'from sklearn.svm import SVC',
   'from sklearn.svm import SVC # "Support vector classifier"',
   'import matplotlib.pyplot as plt',
   'import numpy as np',
   'import seaborn as sns; sns.set()'],
  'headin

# Search Engine widget

In [134]:
import ipywidgets as widgets

In [367]:
# Text 
normal_text = widgets.Text() #description="Search", continuous_update=True)
code = widgets.Text(description="Code")
markdown = widgets.Text(description="Markdown")
modules = widgets.Text(description="Modules")

# Buttons
search_b_normal = widgets.Button(description="Search")
search_b_adv = widgets.Button(description="Search")
clear_b = widgets.Button(description="Clear")

output = widgets.Output()





def on_normal_search_clicked(b):
    with output:
        output.clear_output()
        if normal_text.value == '':
            print("Type a query")
        else:
#             print(query_maker(normal_text.value,normal_text.value,normal_text.value,None))
            display_results(multi_match_query(normal_text.value,normal_text.value,normal_text.value,True),20)

def on_adv_search_clicked(b):
    send_code = code.value
    send_markdown = markdown.value
    send_modules = modules.value
    with output:
        output.clear_output()
        if code.value == "" and markdown.value=='' and modules.value=='':
            print("Type a query")
        else:
            if code.value == '':
                send_code = None
            if markdown.value == '':
                send_markdown = None
            if modules.value == '':
                send_modules = None
            print(query_string_query(send_code,send_markdown,send_modules,True))
            display_results(query_string_query(send_code,send_markdown,send_modules,True),20)

        
def on_clear_clicked(b):
    with output:
        output.clear_output()
    normal_text.value = ''
    markdown.value = ''
    code.value = ''
    modules.value = ''
    
        
def switch(check):
    with output:                      # remove if you want results to stay when
        output.clear_output()         # you switch between normal and adv search
    if check['new']:
        search_display.children=[row_adv,row_output]
    else:
        search_display.children=[row_normal,row_output]

        
def display_results(query,max_res):
    result = es.search(body=query, size=max_res)
    for i in range(len(result['hits']['hits'])): # for all hits
        res = result['hits']['hits'][i]
        modules = res['_source']['modules']
        
        res_layout = widgets.Layout(maring='0px')
        
        title = widgets.HTML(value='<b>'+res['_source']['file_name']+'</b>')#,layout=res_layout)
        url_str = 'https://github.com/jakevdp/PythonDataScienceHandbook/blob/master/notebooks/'+res['_source']['file_name']
        url = widgets.HTML(value='<a href='+url_str + ' target="_blank">' + url_str + '</a>')
        score = widgets.HTML(value=str(res['_score']))#,layout=res_layout)
        
        whole_layout = widgets.Layout(display='flex',
                        flex_flow='column',
                        align_items='stretch',
                        border='grey solid 1px')
        
        
        try:
            highlight_str = str(res['highlight'].values())
            highlight = widgets.HTML(value=highlight_str,layout=res_layout)
            single_result = widgets.VBox([title,score,url,highlight],layout=whole_layout)
#             print(res['highlight'].keys())            
#             print(highlight_str)            
        except:
            single_result = widgets.VBox([title,score,url],layout=whole_layout)
        display(single_result)


In [368]:
search_b_normal.on_click(on_normal_search_clicked)
search_b_adv.on_click(on_adv_search_clicked)
clear_b.on_click(on_clear_clicked)

row_normal = widgets.HBox([normal_text,search_b_normal,clear_b])
row_adv = widgets.HBox([markdown,code,modules,search_b_adv,clear_b])
row_output = widgets.VBox([output])

top_toggle = widgets.Checkbox(description='Advanced Search')
top_toggle.observe(switch, names='value')
display(top_toggle)

normal_text.on_submit(on_normal_search_clicked)
code.on_submit(on_adv_search_clicked)
markdown.on_submit(on_adv_search_clicked)
modules.on_submit(on_adv_search_clicked)


search_display = widgets.VBox(children = [row_normal,row_output])
display(search_display)

Checkbox(value=False, description='Advanced Search')

In [ ]:
# result = es.search(body=query, size=max_res)
#     transcripts = ""
#     for i in range(len(result['hits']['hits'])): # for all hits
#         res = result['hits']['hits'][i]
#         modules = res['_source']['modules']
#         #len(res['_source']['modules'])
#         print(res['_score'],'https://github.com/jakevdp/PythonDataScienceHandbook/blob/master/notebooks/'+res['_source']['file_name'])
#         print(res['highlight']['markdown_str'])

In [253]:
# test_query = {'query': {'multi_match': {'fields': ['modules'], 'query': ' numpy'}}}
# search_results(test_query,20)

enter toets gebruiken voor search:
- link: https://stackoverflow.com/questions/47137370/getting-text-from-jupyter-text-widget
- link: https://pythonprogs.blogspot.com/2017/01/widgets-for-jupyter-notebook-text-input.html

advanced search toggle (advanced search tonen/verbergen):
- link: https://stackoverflow.com/questions/54093955/ipywidgets-use-checkbox-to-display-or-hide-other-widgets